Launch CMD

In [1]:
import os, subprocess
rclone = f"{os.path.expanduser('~/Documents')}/rclone-v1.66.0-windows-amd64"
c = f'-n FR-Gri'
c += f' -lat 48.84422'
c += f' -lon 1.95191'
c += ' -START_DATE 2023-12-22'
c += ' -END_DATE 2024-12-01'
#c += ' --new_band_names NDVI EVI LSWI'
#c += ' --new_band_formulas "(NIR - RED) / (NIR + RED)" "2.5 * (NIR -RED) / ((NIR + 6 * RED - 7.5 * BLUE) + 1 * 10**4)" "(B8 - B11) / (B8 + B11)"'

c += ' --download_folder "Grignon/Satellites/raw/"'
c += ' --download_to_cloud 1'
c += f' --download_rclone_path {rclone}'
#c += ' --download_selected_bands NDVI EVI LSWI'

subprocess.Popen(f'start "ee_google" cmd /k python ee_google.py {c}', shell=True)

Run on notebook

In [ ]:
"""Run Earth Engine extraction"""
import threading
import ee_google as gee

g = gee.eengine(**{
    "name":         'FR-Gri',
    "longitude":    1.95191,
    "latitude":     48.84422,
    "TILE":         '31UDQ',
    "START_DATE":   "2023-12-03",
    "END_DATE":     "2023-12-05",
    #"CLD_PRB_THRESH": CLD_PRB_THRESH,  # %
    #"NIR_DRK_THRESH": NIR_DRK_THRESH,  # ?
    #"CLD_PRJ_DIST": CLD_PRJ_DIST,  # in meters
    "BUFFER":       10**3,  # in meters
}).get_collection('COPERNICUS/S2_SR_HARMONIZED')

# Thread it not to freeze python for long run
# /!\ caution, multiple runs WILL run multiple times in google's servers
#              resulting in duplicates on your drive
threading.Thread(target=g.download,
                 kwargs={"folder": "data/tmp/Grignon", "to_cloud": False}).start()


Visualize

In [1]:
import ee_google as gee

g = gee.eengine(**{
    "name":         'FR-Gri',
    "longitude":    1.95191,
    "latitude":     48.84422,
    "TILE":         '31UDQ',
    "START_DATE":   "2021-12-03",
    "END_DATE":     "2024-12-05",
    "BUFFER":       10**3,  # in meters
}).get_collection('COPERNICUS/S2_SR_HARMONIZED').add_bands()

img_viz_pms = {'image_viz_params': {
                'NDVI composite': {'bands': ['NDVI'], 'min': -10000.0, 'max': 10000.0, 'palette': 'viridis'},
                'true color composite': {'bands': ['TCI_R', 'TCI_G', 'TCI_B'],},
                }}
g.s2_cloud_free(CLD_PRB_THRESH=10).show(collection='COPERNICUS/S2_SR_HARMONIZED_cloudless', zoom=13, **img_viz_pms)


Map(center=[48.84422, 1.95191], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…